# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 100 new papers today
          37 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/36 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.12199


extracting tarball to tmp_2210.12199...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.12207


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.12199/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2210.13249... done.
Retrieving document from  https://arxiv.org/e-print/2210.13260


extracting tarball to tmp_2210.13260...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.13294


extracting tarball to tmp_2210.13294...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.12412-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.12412) | **Impact of water vapor seeing on mid-infrared high-contrast imaging at  ELT scale**  |
|| Olivier Absil, et al. -- incl., <mark>Roy van Boekel</mark> |
|*Appeared on*| *2022-10-25*|
|*Comments*| *13 pages, paper presented at SPIE Astronomical Telescopes + Instrumentation 2022*|
|**Abstract**| The high-speed variability of the local water vapor content in the Earth atmosphere is a significant contributor to ground-based wavefront quality throughout the infrared domain. Unlike dry air, water vapor is highly chromatic, especially in the mid-infrared. This means that adaptive optics correction in the visible or near-infrared domain does not necessarily ensure a high wavefront quality at longer wavelengths. Here, we use literature measurements of water vapor seeing, and more recent infrared interferometric data from the Very Large Telescope Interferometer (VLTI), to evaluate the wavefront quality that will be delivered to the METIS mid-infrared camera and spectrograph for the Extremely Large Telescope (ELT), operating from 3 to 13 {\mu}m, after single-conjugate adaptive optics correction in the near-infrared. We discuss how the additional wavefront error due to water vapor seeing is expected to dominate the wavefront quality budget at N band (8-13 {\mu}m), and therefore to drive the performance of mid-infrared high-contrast imaging modes at ELT scale. Then we present how the METIS team is planning to mitigate the effect of water vapor seeing using focal-plane wavefront sensing techniques, and show with end-to-end simulations by how much the high-contrast imaging performance can be improved. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.12199-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.12199) | **Resolved Molecular Gas Observations of MaNGA Post-starbursts Reveal a  Tumultuous Past**  |
|| Justin Atsushi Otter, et al. -- incl., <mark>Timothy Heckman</mark>, <mark>Lihwai Lin</mark>, <mark>Charles Liu</mark> |
|*Appeared on*| *2022-10-25*|
|*Comments*| *Accepted for publication in The Astrophysical Journal*|
|**Abstract**| Post-starburst galaxies (PSBs) have recently and rapidly quenched their star-formation, thus they are an important way to understand how galaxies transition from star-forming late-types to quiescent early-types. The recent discovery of large cold gas reservoirs in PSBs calls into question the theory that galaxies must lose their gas to become quiescent. Optical Integral Field Spectroscopy (IFS) surveys have revealed two classes of PSBs: central PSBs with central quenching regions and ring PSBs with quenching in their outskirts. We analyze a sample of 13 nearby (z < 0.1) PSBs with spatially resolved optical IFS data from the Mapping Nearby Galaxies at Apache Point Observatory (MaNGA) survey and matched resolution Atacama Large (sub-)Millimeter Array (ALMA) observations of $^{12}$CO(1-0). Disturbed stellar kinematics in 7/13 of our PSBs and centrally concentrated molecular gas is consistent with a recent merger for most of our sample. In galaxies without merger evidence, alternate processes may funnel gas inwards and suppress star-formation, which may include outflows, stellar bars, and minor mergers or interactions. The star-formation efficiencies of the post-starburst regions in nearly half our galaxies are suppressed while the gas fractions are consistent with star-forming galaxies. AGN feedback may drive this stabilization, and we observe AGN-consistent emission in the centers of 5/13 galaxies. Finally, our central and ring PSBs have similar properties except the ionized and molecular gas in central PSBs is more disturbed. Overall, the molecular gas in our PSBs tends to be compact and highly disturbed, resulting in concentrated gas reservoirs unable to form stars efficiently. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.12207-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.12207) | **A New Distance to the Supernova Remnant DA 530 Based on HI Absorption of  Polarized Emission**  |
|| Rebecca A. Booth, et al. -- incl., <mark>Jo-Anne Brown</mark> |
|*Appeared on*| *2022-10-25*|
|*Comments*| **|
|**Abstract**| Supernova remnants (SNRs) are significant contributors of matter and energy to the interstellar medium. Understanding the impact and the mechanism of this contribution requires knowledge of the physical size, energy, and expansion rate of individual SNRs, which can only come if reliable distances can be obtained. We aim to determine the distance to the SNR DA 530 (G93.3+6.9), an object of low surface brightness. To achieve this, we used the Dominion Radio Astrophysical Observatory Synthesis Telescope and the National Radio Astronomy Observatory Very Large Array to observe the absorption by intervening HI of the polarized emission from DA 530. Significant absorption was detected at velocities $-28$ and -67 km/s (relative to the local standard of rest), corresponding to distances of 4.4 and 8.3 kpc, respectively. Based on the radio and X-ray characteristics of DA 530, we conclude that the minimum distance is 4.4$^{+0.4}_{-0.2}$ kpc. At this minimum distance, the diameter of the SNR is 34$^{+4}_{-1}$ pc, and the elevation above the Galactic plane is 537$^{+40}_{-32}$ pc. The $-67$ km/s absorption likely occurs in gas whose velocity is not determined by Galactic rotation. We present a new data processing method for combining Stokes $Q$ and $U$ observations of the emission from an SNR into a single HI absorption spectrum, which avoids the difficulties of the noise-bias subtraction required for the calculation of polarized intensity. The polarized absorption technique can be applied to determine distances to many more SNRs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.13260-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.13260) | **Almanac: Weak Lensing power spectra and map inference on the masked  sphere**  |
|| A. Loureiro, et al. -- incl., <mark>A. F. Heavens</mark> |
|*Appeared on*| *2022-10-25*|
|*Comments*| *18 pages, 16 figures*|
|**Abstract**| We present a field-based signal extraction of weak lensing from noisy observations on the curved and masked sky. We test the analysis on a simulated Euclid-like survey, using a Euclid-like mask and noise level. To make optimal use of the information available in such a galaxy survey, we present a Bayesian method for inferring the angular power spectra of the weak lensing fields, together with an inference of the noise-cleaned tomographic weak lensing shear and convergence (projected mass) maps. The latter can be used for field-level inference with the aim of extracting cosmological parameter information including non-gaussianity of cosmic fields. We jointly infer all-sky $E$-mode and $B$-mode tomographic auto- and cross-power spectra from the masked sky, and potentially parity-violating $EB$-mode power spectra, up to a maximum multipole of $\ell_{\rm max}=2048$. We use Hamiltonian Monte Carlo sampling, inferring simultaneously the power spectra and denoised maps with a total of $\sim 16.8$ million free parameters. The main output and natural outcome is the set of samples of the posterior, which does not suffer from leakage of power from $E$ to $B$ unless reduced to point estimates. However, such point estimates of the power spectra, the mean and most likely maps, and their variances and covariances, can be computed if desired. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.13294-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.13294) | **Quasi-spherical superclusters**  |
|| <mark>Pekka Heinämäki</mark>, et al. |
|*Appeared on*| *2022-10-25*|
|*Comments*| *15 pages, 10 figures, 2 tables, accepted in Astronomy & Astrophysics*|
|**Abstract**| Generally the dynamical state of superclusters is poorly known. We study properties of superclusters and select a sample of quasi-spherical superclusters, the dynamics of which can be studied using the $\Lambda$ significance diagram. We extracted our supercluster sample with an adaptive local threshold density method from the Sloan Digital Sky Survey Data Release 7 (SDSS DR7) data and estimated their masses using the dynamical masses for member galaxies and groups. We used topological analysis based on Minkowski functionals and the positions of galaxies and galaxy groups in superclusters. Finally, we highlight the dynamical state of a few exceptional types of superclusters found in this study using the $\Lambda$ significance diagram. Our final sample contains 65 superclusters in the distance range of 130 to 450 Mpc. Supercluster masses range between $1.1 \times 10^{15} M_{\sun}$ and $1.4 \times 10^{16} M_{\sun}$ and sizes between 25 Mpc and 87 Mpc. We find that pancake-type superclusters form the low-luminosity, small, poor and low-mass end of superclusters. We find four superclusters of unusual types, exhibiting exceptionally spherical shapes. These so-called quasi-spherical systems contain a high-density core surrounded by a relatively spherical density and galaxy distribution.The mass-to-light ratio of these quasi-sphericals is higher than those of the other superclusters, suggesting a relatively high dark matter content. Using the $\Lambda$ significance diagram for oblate and prolate spheroids, we find that three quasi-spherical superclusters are gravitationally bound at the present epoch. Quasi-spherical superclusters are among the largest gravitationally bound systems found to date, and form a special class of giant systems that, dynamically, are in between large gravitationally unbound superclusters and clusters of galaxies in an equilibrium configuration. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.12708-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.12708) | **Mid-Infrared Polarization of the Diffuse Interstellar Medium toward  CygOB2-12**  |
|| Charles M. Telesco, et al. -- incl., <mark>Frank Varosi</mark> |
|*Appeared on*| *2022-10-25*|
|*Comments*| *Accepted Astrophysical Journal Letters, October 9, 2022*|
|**Abstract**| We present the first mid-IR detection of the linear polarization toward the star CygOB2-12, a luminous blue hypergiant that, with AV of ~10 mag of foreground extinction, is a benchmark in the study of the properties of dust in the diffuse interstellar medium. The 8-13 micrometer spectropolarimetry, obtained with the CanariCam multi-mode camera at the Gran Telescopio Canarias (GTC), shows clear trends with wavelength characteristic of silicate grains aligned in the interstellar magnetic field. The maximum polarization, detected with 7.8 statistical significance near 10.2 micrometers, is (1.24 +/- 0.28) % with position angle (126 +/- 8) deg. We comment on these measurements in the context of recent models for the dust composition in the diffuse interstellar medium. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.12710-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.12710) | **L 363-38 b: a planet newly discovered with ESPRESSO orbiting a nearby M  dwarf star**  |
|| <mark>Lia F. Sartori</mark>, et al. |
|*Appeared on*| *2022-10-25*|
|*Comments*| *12 pages, 11 figures*|
|**Abstract**| Context. Planets around stars in the solar neighbourhood will be prime targets for characterisation with upcoming large space- and ground-based facilities. Since large-scale exoplanet searches will not be feasible with such telescopes, it is crucial to use currently available data and instruments to find possible target planets before next generation facilities come online. Aims. We aim at detecting new extrasolar planets around stars in the solar neighbourhood by blind radial velocity (RV) search with ESPRESSO. Our target sample consist of nearby stars (d < 11 pc) with little (< 10) or no previous RV measurements. Methods. We use 31 radial velocity measurements obtained with ESPRESSO at the VLT between December 2020 and February 2022 of the nearby M dwarf star (M_star = 0.21 M_sun, d = 10.23 pc) L 363-38 to derive the orbital parameters of the newly discovered planet. In addition, we use TESS photometry and archival VLT/NaCo high contrast imaging data to put further constraints on the orbit inclination and the possible planetary system architecture around L 363-38. Results. We present the detection of a new extrasolar planet orbiting the nearby M dwarf star L 363-38. L 363-38 b is a planet with minimum mass mp sin(i) = 4.67+/-0.43 M_Earth orbiting its star with a period P = 8.781+/-0.007 d, corresponding to a semi-major axis a = 0.048+/-0.006 AU, which is well inside the inner edge of the habitable zone. We further estimate a minimum radius rp sin(i) = 1.55 - 2.75 R_Earth and an equilibrium temperature Teq = 330K. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.12344-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.12344) | **Systematic KMTNet Planetary Anomaly Search, Paper VII: Complete Sample  of $q < 10^{-4}$ Planets from the First Four-Year Survey**  |
|| Weicheng Zang, et al. -- incl., <mark>Xiangyu Zhang</mark>, <mark>Andrew Gould</mark>, <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark>, <mark>Jiyuan Zhang</mark> |
|*Appeared on*| *2022-10-25*|
|*Comments*| *15 figures and 14 tables; submitted to AAS*|
|**Abstract**| We present the analysis of seven microlensing planetary events with planet/host mass ratios $q < 10^{-4}$: KMT-2017-BLG-1194, KMT-2017-BLG-0428, KMT-2019-BLG-1806, KMT-2017-BLG-1003, KMT-2019-BLG-1367, OGLE-2017-BLG-1806, and KMT-2016-BLG-1105. They were identified by applying the Korea Microlensing Telescope Network (KMTNet) AnomalyFinder algorithm to 2016--2019 KMTNet events. A Bayesian analysis indicates that all the lens systems consist of a cold super-Earth orbiting an M or K dwarf. Together with 17 previously published and three that will be published elsewhere, AnomalyFinder has found a total of 27 planets that have solutions with $q < 10^{-4}$ from 2016--2019 KMTNet events, which lays the foundation for the first statistical analysis of the planetary mass-ratio function based on KMTNet data. By reviewing the 27 planets, we find that the missing planetary caustics problem in the KMTNet planetary sample has been solved by AnomalyFinder. We also find a desert of high-magnification planetary signals ($A \gtrsim 65$), and a follow-up project for KMTNet high-magnification events could detect at least two more $q < 10^{-4}$ planets per year and form an independent statistical sample. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.12412.md
    + _build/html/tmp_2210.12412/./WV_L_no_tiptilt.png
    + _build/html/tmp_2210.12412/./WV_N2_no_tiptilt.png
    + _build/html/tmp_2210.12412/./n_wda.png
    + _build/html/tmp_2210.12412/./berio_fig.png
    + _build/html/tmp_2210.12412/./time_sequence.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\baselinestretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\baselinestretch$}{1.0}$</div>



<div id="title">

# Impact of water vapor seeing on mid-infrared high-contrast imaging at ELT scale

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.12412-b31b1b.svg)](https://arxiv.org/abs/2210.12412)<mark>Appeared on: 2022-10-25</mark> - _13 pages, paper presented at SPIE Astronomical Telescopes + Instrumentation 2022_

</div>
<div id="authors">

Olivier Absil, et al. -- incl., <mark>Roy~van~Boekel</mark>

</div>
<div id="abstract">

**Abstract:** The high-speed variability of the local water vapor content in the Earth atmosphere is a significant contributor to ground-based wavefront quality throughout the infrared domain. Unlike dry air, water vapor is highly chromatic, especially in the mid-infrared. This means that adaptive optics correction in the visible or near-infrared domain does not necessarily ensure a high wavefront quality at longer wavelengths. Here, we use literature measurements of water vapor seeing, and more recent infrared interferometric data from the Very Large Telescope Interferometer (VLTI), to evaluate the wavefront quality that will be delivered to the METIS mid-infrared camera and spectrograph for the Extremely Large Telescope (ELT), operating from 3 to 13$\mu$m, after single-conjugate adaptive optics correction in the near-infrared. We discuss how the additional wavefront error due to water vapor seeing is expected to dominate the wavefront quality budget at N band (8--13$\mu$m), and therefore to drive the performance of mid-infrared high-contrast imaging modes at ELT scale. Then we present how the METIS team is planning to mitigate the effect of water vapor seeing using focal-plane wavefront sensing techniques, and show with end-to-end simulations by how much the high-contrast imaging performance can be improved.

</div>

<div id="div_fig1">

<img src="tmp_2210.12412/./WV_L_no_tiptilt.png" alt="Fig8.1" width="50%"/><img src="tmp_2210.12412/./WV_N2_no_tiptilt.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** example\label{fig:perf_kolmo}
Post-ADI contrast curves for a 1-h observing sequence at L band (top) and N band (bottom), in the presence of SCAO residuals only (black), and of SCAO residual and WV seeing (blue). The orange curves show the performance after completely removing tip-tilt errors from the WV phase screens. These contrast curves were obtained with a ring-apodized vortex coronagraph (RAVC) concept at L band, and a classical vortex coronagraph (CVC) concept at N band, following the METIS design\cite{Carlomagno20}. (*fig:perf_kolmo*)

</div>
<div id="div_fig2">

<img src="tmp_2210.12412/./n_wda.png" alt="Fig2" width="100%"/>

**Figure 2. -** example\label{fig:refrac_wda}
Reduced refractive index of WDA, for standard conditions at Cerro Paranal (288 K, 743 mbar). (*fig:refrac_wda*)

</div>
<div id="div_fig3">

<img src="tmp_2210.12412/./berio_fig.png" alt="Fig4.1" width="50%"/><img src="tmp_2210.12412/./time_sequence.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** example\label{fig:gra4mat}
Left. Comparison of the N-band fringe position estimated by the GRAVITY fringe tracker (red) with the actually measured fringe position with MATISSE (blue), adapted from RD08. Right. Representative time sequence of differential WV column density measured on a $\sim 30$-m baseline at VLTI. (*fig:gra4mat*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

73  publications in the last 7 days.
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers